## Get HTML

In [60]:
from urllib2 import urlopen
endpoint = "http://www.tigerjython.ch"
response = urlopen(endpoint)
html = response.read()
print(html)

<html>
<body>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">
<meta HTTP-EQUIV="REFRESH" content="0; url=index.php?inhalt_links=navigation.inc.php&inhalt_mitte=home/home.inc.php">
</body>
</html>


## GET HTML mit Encoding

In [65]:
from urllib2 import urlopen
endpoint = "http://www.tigerjython.ch/index.php?inhalt_links=navigation.inc.php&inhalt_mitte=robotik/lernendeRobot.inc.php"
response = urlopen(endpoint)
html = response.read()
encodedhtml = unicode(html, 'iso-8859-1')
#print(html)
print(encodedhtml)

<!DOCTYPE html lang="de">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">
<meta name="page-topic" content="TigerJython">
<meta name="keywords" content="python, jython, programmieren, python-editor, python IDE, tutorial, lehrmittel, turtlegrafik, robotik, games">
<meta name="audience" content="Gymnasiallehrer, Sch�lerinnen, Sch�ler Sekundarstufe 1, Sekundarschule 2, Programmieranf�nger">
<meta name="description" content="Einstieg ins Programmieren mit Python, Python IDE, Turtlegrafik, Robotik, Games, Grafik, Online-Lehrmittel ">
<meta name="robots" content="index, follow">
<meta name="viewport" content="width=1024" />
<link rel ="shortcut icon" href = "tigerjython.ico">
<link href="tigerjython.css" rel="stylesheet" type="text/css">
<script type="text/javascript">
var txt = "neuer <b>fetter<\/b> Text";

function selectText(element) 
{
    var doc = document;
    var text = doc.getElementById(element);    

    if (doc.body.createTextRange)  // ms
	{ 
     

## GET RESPONSE.Headers & HTTP STATUS

In [61]:
from urllib2 import urlopen

endpoint = "http://www.tigerjython.ch"
endpoint = "http://www.tigerjython.ch/index.php?inhalt_links=navigation.inc.php&inhalt_mitte=lernumgebung/lernumgebung.inc.php"

handler = urlopen(endpoint)
headers = handler.headers
code = handler.getcode()
msg = handler.msg
handler.close()

print("HTTP-Status:",code)
print(msg)
print(headers)
print handler.headers['content-type']

('HTTP-Status:', 200)
OK
Date: Wed, 26 Aug 2015 15:53:34 GMT
Server: Apache
Content-Type: text/html
X-Cache: MISS from login.fdxtended.com
Via: 1.0 login.fdxtended.com (squid/3.0.STABLE20)
Proxy-Connection: close

text/html


## JSON Informationen über github User mgje

In [32]:
import json
from urllib2 import urlopen

endpoint = "https://api.github.com/users/mgje"
handler = urlopen(endpoint)

j_obj = json.loads(handler.read())

userid = j_obj['login']
name = j_obj['name']
created_at = j_obj['created_at']

print('GitHub user ',userid)
print('Name: ',name)
print('since: ',created_at)

handler.close()


('GitHub user ', u'mgje')
('Name: ', u'Martin Guggisberg')
('since: ', u'2012-04-05T20:54:13Z')


## HTML Parsen

In [37]:
from HTMLParser import HTMLParser
from urllib2 import urlopen

# create a subclass and override the handler methods
class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print "Encountered a start tag:", tag
    def handle_endtag(self, tag):
        print "Encountered an end tag :", tag
    def handle_data(self, data):
        print "Encountered some data  :", data


endpoint = "http://www.tigerjython.ch"
response = urlopen(endpoint)
html = response.read()
print(html)

# instantiate the parser and fed it some HTML
parser = MyHTMLParser()
parser.feed(html)

<html>
<body>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">
<meta HTTP-EQUIV="REFRESH" content="0; url=index.php?inhalt_links=navigation.inc.php&inhalt_mitte=home/home.inc.php">
</body>
</html>
Encountered a start tag: html
Encountered some data  : 

Encountered a start tag: body
Encountered some data  : 

Encountered a start tag: meta
Encountered some data  : 

Encountered a start tag: meta
Encountered some data  : 

Encountered an end tag : body
Encountered some data  : 

Encountered an end tag : html


## more complex Parser

In [38]:
from HTMLParser import HTMLParser
from htmlentitydefs import name2codepoint
from urllib2 import urlopen

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print "Start tag:", tag
        for attr in attrs:
            print "     attr:", attr
    def handle_endtag(self, tag):
        print "End tag  :", tag
    def handle_data(self, data):
        print "Data     :", data
    def handle_comment(self, data):
        print "Comment  :", data
    def handle_entityref(self, name):
        c = unichr(name2codepoint[name])
        print "Named ent:", c
    def handle_charref(self, name):
        if name.startswith('x'):
            c = unichr(int(name[1:], 16))
        else:
            c = unichr(int(name))
        print "Num ent  :", c
    def handle_decl(self, data):
        print "Decl     :", data

        
endpoint = "http://www.tigerjython.ch"
response = urlopen(endpoint)
html = response.read()
print(html)

parser = MyHTMLParser()
parser.feed(html)

<html>
<body>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">
<meta HTTP-EQUIV="REFRESH" content="0; url=index.php?inhalt_links=navigation.inc.php&inhalt_mitte=home/home.inc.php">
</body>
</html>
Start tag: html
Data     : 

Start tag: body
Data     : 

Start tag: meta
     attr: ('http-equiv', 'Content-Type')
     attr: ('content', 'text/html; charset=iso-8859-1')
Data     : 

Start tag: meta
     attr: ('http-equiv', 'REFRESH')
     attr: ('content', '0; url=index.php?inhalt_links=navigation.inc.php&inhalt_mitte=home/home.inc.php')
Data     : 

End tag  : body
Data     : 

End tag  : html


## Nach bestimmten Inhalten Parsen
        

In [42]:
from HTMLParser import HTMLParser


class AllLanguages(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.inLink = False
        self.dataArray = []
        self.countLanguages = 0
        self.lasttag = None
        self.lastname = None
        self.lastvalue = None

    def handle_starttag(self, tag, attrs):
        self.inLink = False
        if tag == 'a':
            for name, value in attrs:
                if name == 'class' and value == 'Vocabulary':
                    self.countLanguages += 1
                    self.inLink = True
                    self.lasttag = tag

    def handle_endtag(self, tag):
        if tag == "a":
            self.inlink = False

    def handle_data(self, data):
        if self.lasttag == 'a' and self.inLink and data.strip():
            print data


parser = AllLanguages()
parser.feed("""
<html>
<head><title>Test</title></head>
<body>
<a href="http://wold.livingsources.org/vocabulary/1" title="Swahili" class="Vocabulary">Swahili</a>
<a href="http://wold.livingsources.org/contributor#schadebergthilo" title="Thilo Schadeberg" class="Contributor">Thilo Schadeberg</a>
<a href="http://wold.livingsources.org/vocabulary/2" title="English" class="Vocabulary">English</a>
<a href="http://wold.livingsources.org/vocabulary/2" title="Russian" class="Vocabulary">Russian</a>
</body>
</html>""")

Swahili
English
Russian


## Parser Example

In [50]:
from HTMLParser import HTMLParser
from urllib2 import urlopen

class MyParse(HTMLParser):
    def __init__(self):
        #super() does not work for this class
        HTMLParser.__init__(self)
        self.tag_stack = []
        self.attr_stack = []

    def handle_endtag(self, tag):
        #take the tag off the stack if it matches the next close tag
        #if you are expecting unmatched tags, then this needs to be more robust
        if self.tag_stack[len(self.tag_stack)-1][0] == tag:
            self.tag_stack.pop()

    def handle_data(self, data):
        #'data' is the text between tags, not necessarily
        #matching tags
        #this gives you a link to the last tag
        if len(self.tag_stack) > 0:
            tstack = self.tag_stack[len(self.tag_stack)-1]
        else:
            tstack = 0
            
        print(tstack,data)
        #do something with the text
            
    def handle_starttag(self, tag, attrs):
        #add tag to the stack
        self.tag_stack.append([tag, attrs])
        #if this tag is a link
        if tag =="a":
            #these next few lines find if there is a hyperlink in the tag
            tloc = map(lambda x: 1 if x[0]=='href' else 0,attrs)
            try:
                #did we find any hyperlinks
                attr_loc = tloc.index(1)
            except:
                pass
            # attr_loc only exists if we found a hyperlink
            if vars().has_key('attr_loc'):
                #append to the last item in the stack the location of the hyperlink
                #note, this does not increase the length of the stack
                #as we are putting it inside the last item on the stack
                self.tag_stack[len(self.tag_stack)-1].append(attr_loc)

endpoint = "http://curioussystem.com"
response = urlopen(endpoint)
html = response.read()
#print(html)
                
p = MyParse()
p.feed(html)  

(0, '\r\n\r\n')
(['html', [('xmlns', 'http://www.w3.org/1999/xhtml'), ('xml:lang', 'en-gb'), ('lang', 'en-gb')]], '\r\n\r\n')
(['head', []], '\r\n    ')
(['script', [('type', 'text/javascript')]], "\r\nvar siteurl='/';\r\nvar tmplurl='/templates/ja_purity_ii/';\r\nvar isRTL = false;\r\n")
(['head', []], '\r\n\r\n  ')
(['head', []], '\n  ')
(['head', []], '\n  ')
(['head', []], '\n  ')
(['title', []], 'Home - Curious System Solutions, LLC')
(['head', []], '\n  ')
(['head', []], '\n  ')
(['head', []], '\n  ')
(['head', []], '\n  ')
(['head', []], '\n  ')
(['script', [('type', 'text/javascript')]], "\nwindow.addEvent('load', function() {\n\t\t\t\tnew JCaption('img.caption');\n\t\t\t});\n  ")
(['head', []], '\n\n')
(['head', []], ' \n')
(['head', []], ' \n')
(['head', []], ' \n\r\n\r\n\r\n\r\n')
(['head', []], '\r\n\r\n\r\n')
(['head', []], '\r\n\r\n')
(['script', [('language', 'javascript'), ('type', 'text/javascript')]], "\r\nvar rightCollapseDefault='show';\r\nvar excludeModules='38';\r

## Link Parser

In [57]:
from HTMLParser import HTMLParser
from urllib2 import urlopen


class LinksExtractor(HTMLParser): # derive new HTML parser

    def __init__(self) :        # class constructor
        HTMLParser.__init__(self)  # base class constructor
        self.links = []        # create an empty list for storing hyperlinks

    def start_a(self, attrs) :  # override handler of <A ...>...</A> tags
      # process the attributes
      if len(attrs) > 0 :
        for attr in attrs :
            if attr[0] == "href" :         # ignore all non HREF attributes
                self.links.append(attr[1]) # save the link info in the list

    def get_links(self) :     # return the list of extracted links
        return self.links
    

htmlparser = LinksExtractor() 

endpoint = "http://cis.poly.edu/index.htm"
response = urlopen(endpoint)
html = response.read()

htmlparser.feed(data.read())      # parse the file saving the info about links
htmlparser.close()

links = htmlparser.get_links()   # get the hyperlinks list
print links   # print all the links
    

HTTPError: HTTP Error 404: Not Found

## HTML Parsen mit BeautifulSoup

In [33]:
from bs4 import BeautifulSoup
import urllib2

response = urllib2.urlopen("http://www.pythonscraping.com/pages/page3.html")
html = response.read()
bsObj = BeautifulSoup(html)

for sibling in bsObj.find("table",{"id":"giftList"}).tr.next_siblings:
    print(sibling) 



<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>
</td></tr>


<tr class="gift" id="gift4"><td>
Dead Parrot
</td><td>
This is an ex-parr

In [13]:
import re

images = bsObj.findAll("img", {"src":re.compile("\.\.\/img\/gifts/img.*\.jpg")})
for image in images: 
    print(image["src"])

../img/gifts/img1.jpg
../img/gifts/img2.jpg
../img/gifts/img3.jpg
../img/gifts/img4.jpg
../img/gifts/img6.jpg


### WikiPedia by Random

In [4]:
import urllib2
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())

def getLinks(articleUrl):
    res = urllib2.urlopen("http://de.wikipedia.org"+articleUrl)
    html = res.read()
    bsObj = BeautifulSoup(html)
    return bsObj.find("div", {"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$"))

links = getLinks("/wiki/Basel")
while len(links) > 15:
    newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
    print(newArticle)
    links = getLinks(newArticle)


/wiki/Basler_Kantonalbank
/wiki/Kantonalbank
/wiki/%C3%96ffentlich-rechtliches_Kreditinstitut
/wiki/Europ%C3%A4ische_Kommission
/wiki/Simone_Veil
/wiki/Jurist
/wiki/Juristenausbildung_in_der_Schweiz


### get Country

In [6]:
import json
import urllib2


def getCountry(ipAddress):
    response = urllib2.urlopen("http://freegeoip.net/json/"+ipAddress).read().decode('utf-8')
    responseJson = json.loads(response)
    return responseJson.get("country_code")

print(getCountry("50.78.253.58"))
print(getCountry("131.152.1.1"))

US
CH


### Wiki History

In [56]:
import urllib2
import json
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())
def getLinks(articleUrl):
    res = urllib2.urlopen("http://de.wikipedia.org"+articleUrl)
    html = res.read()
    bsObj = BeautifulSoup(html)
    return bsObj.find("div", {"id":"bodyContent"}).findAll("a", 
                     href=re.compile("^(/wiki/)((?!:).)*$"))

def getHistoryIPs(pageUrl):
    #Format of revision history pages is: 
    #http://en.wikipedia.org/w/index.php?title=Title_in_URL&action=history
    pageUrl = pageUrl.replace("/wiki/", "")
    historyUrl = "http://de.wikipedia.org/w/index.php?title="+pageUrl+"&action=history"
    print("history url is: "+historyUrl)
    res = urllib2.urlopen(historyUrl)
    html = res.read()
    bsObj = BeautifulSoup(html)
    #finds only the links with class "mw-anonuserlink" which has IP addresses 
    #instead of usernames
    ipAddresses = bsObj.findAll("a", {"class":"mw-anonuserlink"})
    addressList = set()
    for ipAddress in ipAddresses:
        addressList.add(ipAddress.get_text())
    return addressList


def getCountry(ipAddress):
    try:
        response = urllib2.urlopen("http://freegeoip.net/json/"
                           +ipAddress).read().decode('utf-8')
    except HTTPError:
        return None
    responseJson = json.loads(response)
    return responseJson.get("country_code")
    
links = getLinks("/wiki/Basel")


while(len(links) > 13):
    for link in links:
        print("-------------------") 
        historyIPs = getHistoryIPs(link.attrs["href"])
        for historyIP in historyIPs:
            country = getCountry(historyIP)
            if country is not None:
                print(historyIP+" is from "+country)

    newLink = links[random.randint(0, len(links)-1)].attrs["href"]
    links = getLinks(newLink)

-------------------
history url is: http://de.wikipedia.org/w/index.php?title=Basel_(Begriffskl%C3%A4rung)&action=history
79.246.164.59 is from DE
129.132.211.197 is from CH
91.6.246.237 is from DE
88.72.49.214 is from DE
88.72.33.85 is from DE
217.224.254.59 is from DE
85.1.163.19 is from CH
217.230.59.78 is from DE
-------------------
history url is: http://de.wikipedia.org/w/index.php?title=Schweiz&action=history
-------------------
history url is: http://de.wikipedia.org/w/index.php?title=Kanton_(Schweiz)&action=history
84.74.177.35 is from CH
2a02:908:fd32:9900:651f:1bbd:8e48:668 is from DE
92.106.225.204 is from CH
5.149.18.72 is from CH
149.172.245.126 is from DE
-------------------
history url is: http://de.wikipedia.org/w/index.php?title=Kanton_Basel-Stadt&action=history
178.196.243.56 is from CH
194.153.189.154 is from CH
94.198.248.247 is from CH
83.76.228.210 is from CH
92.76.199.49 is from DE
2002:b253:b780:0:3040:7c5c:a696:9e6e is from CH
109.90.123.147 is from DE
-------

NameError: global name 'HTTPError' is not defined

## HTML Parser

In [1]:
from HTMLParser import HTMLParser

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Encountered a start tag:", tag)
    def handle_endtag(self, tag):
        print("Encountered an end tag :", tag)
    def handle_data(self, data):
        print("Encountered some data  :", data)

parser = MyHTMLParser()
parser.feed('<html><head><title>Test</title></head>'
            '<body><h1>Parse me!</h1></body></html>')

('Encountered a start tag:', 'html')
('Encountered a start tag:', 'head')
('Encountered a start tag:', 'title')
('Encountered some data  :', 'Test')
('Encountered an end tag :', 'title')
('Encountered an end tag :', 'head')
('Encountered a start tag:', 'body')
('Encountered a start tag:', 'h1')
('Encountered some data  :', 'Parse me!')
('Encountered an end tag :', 'h1')
('Encountered an end tag :', 'body')
('Encountered an end tag :', 'html')


In [58]:
from HTMLParser import HTMLParser
import urllib2



class MyHTMLParser(HTMLParser):
    counter = 0 
    
    def handle_starttag(self, tag, attrs):
        if self.counter <10:
            print("Encountered a start tag:", tag)
            self.counter +=1
    def handle_endtag(self, tag):
        if self.counter <10:
            print("Encountered an end tag :", tag)
            self.counter +=1
    def handle_data(self, data):
        if self.counter <10:
            print("Encountered some data  :", data)
            self.counter +=1

res = urllib2.urlopen("http://www.nzz.ch")        
encoding = res.headers.getparam('charset')
html = res.read().decode(encoding)             
parser = MyHTMLParser()
parser.feed(html)

('Encountered some data  :', u' ')
('Encountered a start tag:', u'html')
('Encountered some data  :', u' ')
('Encountered a start tag:', u'head')
('Encountered a start tag:', u'meta')
('Encountered an end tag :', u'meta')
('Encountered some data  :', u' ')
('Encountered a start tag:', u'title')
('Encountered some data  :', u'NZZ News \u2013 aktuelle Nachrichten, Hintergr\xfcnde, Kommentare')
('Encountered an end tag :', u'title')


In [10]:
from HTMLParser import HTMLParser
import urllib2

class MyHTMLParser(HTMLParser):
    
    catch = 0
    catchline = ""
    
    def handle_starttag(self, tag, attrs):
        #print("Encountered a start tag:", tag)
        for attr in attrs:
            #print("Attr", attr)
            #print(attr[1])
            if attr[1] == "title__catchline":
                self.catch = 2
            elif attr[1] == "title__name":
                self.catch = 1
            else:
                self.catch = 0
            
#    def handle_endtag(self, tag):
        #print("Encountered an end tag :", tag)
    
    def handle_data(self, data):
        if self.catch==1:
            print self.catchline+": "+data
            self.catchline = ""
            
        if self.catch==2:
            self.catchline = data

res = urllib2.urlopen("http://www.nzz.ch")        
encoding = res.headers.getparam('charset')
html = res.read().decode(encoding)             
parser = MyHTMLParser()
parser.feed(html)

Zwischen Goldstandard und EZB: Wem gehört der Euro?
Sparprogramm im Parlament: Streiks und Proteste in Griechenland
: Was heute wichtig ist
68. Filmfestival Locarno: Ein Programm rund um Häuser
Kampf gegen Boko Haram: Der zögerliche Präsident
Korruptionsverdacht: «Boxenstopp» für den Chef der Tessiner Steuerkontrolle
Auschwitz-Prozess in Lüneburg: Vier Jahre Haft für früheren SS-Mann Gröning
«Wirtschaft im Fokus» mit Marc Parmentier: «Wer gut schlafen will, sollte Festhypotheken abschliessen»
Philosophie des Anti-Aging: Was, wenn wir vor lauter Langeweile sterben möchten?
Amerikanische Esskultur in der Schweiz: Schöne neue Konsumwelt
Vor der Abstimmung im griechischen Parlament: Tsipras gibt sich kämpferisch
Griechische Schuldenlast nicht tragbar: Tiefer Graben zwischen dem IMF und Europa
Griechenland-Deal: Das hätte man billiger haben können
Seitenblick: Im Umgang mit Rettungsschwimmern
Parteipolitische Kakofonie: Adieu, Konkordanz
Vereinbarung mit Iran: Eine Atempause, keine Atompaus

## Rubriken NZZ

In [51]:
from HTMLParser import HTMLParser
import urllib2

class MyHTMLParser(HTMLParser):
    
    rubrik = 0
    catch = 0
    tmp = 0
    
    
    def handle_starttag(self, tag, attrs):
        #print("Encountered a start tag:", tag)
        for attr in attrs:
            #print("Attr", attr)
            #print(attr[1])
            if attr[1]!=None and attr[1].find("container__title ")!=-1 :
                #print("Rubrik")
                self.rubrik += 1
                self.catch = 1
            else:
                #self.catch = 0
                self.tmp=0
    
    def handle_endtag(self, tag):
        if tag == "a" or tag == "div":
            self.catch=0
    
    def handle_data(self, data):
        if self.catch==1:
            if len(data) > 2:
                print(str(self.rubrik)+". "+data)
            

res = urllib2.urlopen("http://www.nzz.ch")        
encoding = res.headers.getparam('charset')
html = res.read().decode(encoding)             
parser = MyHTMLParser()
parser.feed(html)

1. Entscheidungstage für Griechenland
2. Meinung und Debatte
3. International
4. Wirtschaft
5. Finanzen
6. Schweiz
7. NZZ-Spezial zu den Wahlen 2015
8. Feuilleton
9. Zürich
10. Sport
11. Digital
12. Video
13. Panorama
14. Lebensart
15. Medien
16. Wissenschaft
17. Aktuelle Bildstrecken
18. Bild des Tages


###Find Last Article

In [52]:
from HTMLParser import HTMLParser
import urllib2

class MyHTMLParser(HTMLParser):
    
    newesttime = 0
    title = ""
    tmp = 0
    catch=0
    
    
    def handle_starttag(self, tag, attrs):
        #print("Encountered a start tag:", tag)
        if tag == "div":
            for attr in attrs:
                if attr[1] == "title__name":
                    self.catch = 1
            
        
        if tag == "time":
            for attr in attrs:
            #print("Attr", attr)
            #print(attr[1])
                if attr[1]!=None and attr[0].find("datetime")!=-1 :
                    print attr[1]
                

    
    def handle_endtag(self, tag):
        if tag == "a" or tag == "div":
            self.catch=0
    
    def handle_data(self, data):
        if self.catch==1:
            if len(data) > 2:
                print(data)
            

res = urllib2.urlopen("http://www.nzz.ch")        
encoding = res.headers.getparam('charset')
html = res.read().decode(encoding)             
parser = MyHTMLParser()
parser.feed(html)

Wem gehört der Euro?
2015-07-15 05:30:00
Garstiges Umfeld für Sparer und Anleger
2015-07-15 05:30:00
Was heute wichtig ist
2015-07-15 09:40:00
Piccards Solar Impulse fliegt erst 2016 wieder
2015-07-15 14:18:00
Fehdehandschuh der schottischen Nationalisten
2015-07-15 14:39:00
Vier Jahre Haft für früheren SS-Mann Gröning
2015-07-15 10:58:00
Streiks und Proteste in Griechenland
2015-07-15 12:33:00
«Boxenstopp» für den Chef der Tessiner Steuerkontrolle
2015-07-15 13:24:00
2015-07-15 14:43:41
2015-07-15 14:31:11
2015-07-15 14:28:46
2015-07-15 14:21:17
2015-07-15 14:19:59
Neue Hitzewelle gefährdet Feuerwerke am 1. August
2015-07-15 04:00:00
«Wer gut schlafen will, sollte Festhypotheken abschliessen»
2015-07-15 06:00:00
Was, wenn wir vor lauter Langeweile sterben möchten?
2015-07-15 05:30:00
Tsipras gibt sich kämpferisch
2015-07-15 08:10:00
Tiefer Graben zwischen dem IMF und Europa
2015-07-15 05:17:00
Das hätte man billiger haben können
2015-07-14 21:32:00
Im Umgang mit Rettungsschwimmern
201

## BAZ feature Story

In [76]:
from HTMLParser import HTMLParser
import urllib2

class MyHTMLParser(HTMLParser):
    
    rubrik = 0
    catch = 0
    tmp = 0
    
    
    def handle_starttag(self, tag, attrs):
        if tag == "div":
            for attr in attrs:
                if attr[1]!=None and attr[1].find("featureStory")!=-1 :
                    self.rubrik += 1
                    self.catch = 1
                    #print(tag)
                    break
        if tag == "a" and self.catch ==1:
            self.catch =2
    
    def handle_endtag(self, tag):
        if tag == "a":
            self.catch=0
    
    def handle_data(self, data):
        if self.catch==2:
            if len(data) > 1 and data != "  ":
                print(str(self.rubrik)+". "+data)
            

res = urllib2.urlopen("http://bazonline.ch/")        
encoding = res.headers.getparam('charset')
html = res.read().decode(encoding)             
parser = MyHTMLParser()
parser.feed(html)

bla:45
1. «Der Staat unterstützt so einen Schwarzmarkt»


### Baz Rubriken

In [77]:
from HTMLParser import HTMLParser
import urllib2

class MyHTMLParser(HTMLParser):
    
    rubrik = 0
    catch = 0
    tmp = 0
    
    
    def handle_starttag(self, tag, attrs):
        if tag == "div":
            for attr in attrs:
                if attr[1]!=None and attr[1].find("ressortGroup")!=-1 :
                    self.rubrik += 1
                    self.catch = 1
                    #print(tag)
                    break
        if tag == "a" and self.catch ==1:
            self.catch =2
    
    def handle_endtag(self, tag):
        if tag == "a":
            self.catch=0
    
    def handle_data(self, data):
        if self.catch==2:
            if len(data) > 1 and data != "  ":
                print(str(self.rubrik)+". "+data)
            

res = urllib2.urlopen("http://bazonline.ch/")        
encoding = res.headers.getparam('charset')
html = res.read().decode(encoding)             
parser = MyHTMLParser()
parser.feed(html)

1. Basel
2. Schweiz
3. Ausland
4. Wirtschaft
5. Sport
6. Leben
7. Kultur
8. Digital
9. Blogs
10. Panorama


###Baz Artikelliste nach Zeit

In [115]:
from HTMLParser import HTMLParser
import urllib2

class MyHTMLParser(HTMLParser):
    
    rubrik = 0
    catch = 0
    tmp = 0
    storyId = ""
    title = ""
    message = ""
    info =[]
    

    def handle_starttag(self, tag, attrs):
        if tag == "div":
            for attr in attrs:
                if attr[1]!=None and attr[1].find("articleStory_")!=-1 :
                    self.storyId=attr[1]
                    self.rubrik += 1
                    self.info.append(self.rubrik)
                    self.info.append(attr[1])
                    self.catch = 1
                    #print(tag)
                    break
                    
        if tag == "a" and self.catch ==1:
            self.catch = 2
        
        if tag =="h5" and self.catch == 3:
            self.catch = 4
        if tag =="em" and self.catch == 4:
            self.catch = 5
        if tag =="span" and self.catch == 5:
            self.catch = 7
    
    def handle_endtag(self, tag):
        if tag == "a":
            self.catch=3
            
        if tag == "em" and self.catch == 5 or self.catch == 6:
            self.catch=0
            self.info=[]
    
    def handle_data(self, data):
        if self.catch == 2:
            if len(data) > 1 and data != "  ":
                self.title=data
                #self.info.append(self.storyId)
                self.info.append(data)
        elif self.catch == 5:
            if data.find("\t\t\t\t") == -1 and data != " ":
                self.message = str(self.rubrik)+". "+self.storyId+":"+self.title+", "+data
                self.info.append(data)
                print self.info
        elif self.catch == 7:
            self.info.append(data)
            print data
            print("catched")
            

res = urllib2.urlopen("http://bazonline.ch/")        
encoding = res.headers.getparam('charset')
html = res.read().decode(encoding)             
parser = MyHTMLParser()
parser.feed(html)

[u'Aktualisiert um 12:13']
[(u'id', u'aktualisiert-17324495-4')]
[1, u'articleStory_17324495_4714', u'\xabDeutschland ist zu gross f\xfcr Europa\xbb']
 
catched

                    
catched
[1, u'articleStory_17324495_4714', u'\xabDeutschland ist zu gross f\xfcr Europa\xbb', u' ', u'\n                    ', 2, u'articleStory_30391905_3002', u"Tsipras' Regierung droht an ihrem Sieg zu zerbrechen", u'Aktualisiert um 09:03']
[(u'id', u'aktualisiert-10521925-1')]
[3, u'articleStory_10521925_3773', u'Dank Millard\xe4r erfolgreich gegen die SVP-Initiative']
 
catched

                    
catched
[3, u'articleStory_10521925_3773', u'Dank Millard\xe4r erfolgreich gegen die SVP-Initiative', u' ', u'\n                    ', 4, u'articleStory_14730260_8684', u'Schweiz liefert Fifa-Funktion\xe4r an USA aus', u'Aktualisiert um 11:55']
[5, u'articleStory_18259665_8660', u'SBB beschenken Halbtax-Besitzer', u'Aktualisiert um 11:49']
[6, u'articleStory_31774018_7462', 7, u'articleStory_18714754_7851'

In [88]:
p = []
p.append("hallo")
p

['hallo']